In [10]:
import torch
from torch import Tensor
from torch import nn
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

In [84]:
df = pd.read_excel("GNN_data_f.xlsx", engine='openpyxl', index_col=False)
df = df.drop(df.columns[[0]], axis=1)  
df

,Chr_x,Gene symbol,loc,z-score,int_size,eqtl_genes,DISEASE/TRAIT,l_break,r_break,seq
0,chr19,-,47774772,1.641685e+07,NaN,"['ENSG00000105321', 'ENSG00000197405', 'ENSG00...",NaN,47774272,47775272,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...
1,chr18,SMAD4,48609968,4.044151e+06,NaN,"['ENSG00000141646', 'ENSG00000082212', 'ENSG00...",NaN,48609468,48610468,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...
2,chr18,SMAD4,48610474,4.044151e+06,NaN,NaN,NaN,48609974,48610974,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...
3,chr1,-,212459056,1.049522e+06,412.0,NaN,NaN,212458556,212459556,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...
4,chr15,NR2F2,96873540,9.413342e+05,NaN,NaN,NaN,96873040,96874040,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...
...,...,...,...,...,...,...,...,...,...,...
7785,chr12,BCAT1,24965636,9.017178e+00,NaN,NaN,NaN,24965136,24966136,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...
7786,chr4,-,83345632,8.958757e+00,NaN,['ENSG00000145293'],NaN,83345132,83346132,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...
7787,chr12,-,27182338,8.381627e+00,NaN,"['ENSG00000111790', 'ENSG00000152944', 'ENSG00...",NaN,27181838,27182838,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...
7788,chr12,VEZT,95696030,8.337806e+00,NaN,NaN,NaN,95695530,95696530,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...


In [85]:
df["target"] = ((df["z-score"] >= 200) & (~df["eqtl_genes"].isna())) * 1
df["target"].sum()

1215

In [86]:
# df[~df["DISEASE/TRAIT"].isna()]
df['DISEASE/TRAIT'] = df['DISEASE/TRAIT'].apply(lambda l: [s.lower().replace('-', ' ').replace('_', ' ').strip() \
                                                           for s in eval(l)] if isinstance(l, str) else [])
df

,Chr_x,Gene symbol,loc,z-score,int_size,eqtl_genes,DISEASE/TRAIT,l_break,r_break,seq,target
0,chr19,-,47774772,1.641685e+07,NaN,"['ENSG00000105321', 'ENSG00000197405', 'ENSG00...",[],47774272,47775272,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...,1
1,chr18,SMAD4,48609968,4.044151e+06,NaN,"['ENSG00000141646', 'ENSG00000082212', 'ENSG00...",[],48609468,48610468,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...,1
2,chr18,SMAD4,48610474,4.044151e+06,NaN,NaN,[],48609974,48610974,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...,0
3,chr1,-,212459056,1.049522e+06,412.0,NaN,[],212458556,212459556,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...,0
4,chr15,NR2F2,96873540,9.413342e+05,NaN,NaN,[],96873040,96874040,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...,0
...,...,...,...,...,...,...,...,...,...,...,...
7785,chr12,BCAT1,24965636,9.017178e+00,NaN,NaN,[],24965136,24966136,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...,0
7786,chr4,-,83345632,8.958757e+00,NaN,['ENSG00000145293'],[],83345132,83346132,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...,0
7787,chr12,-,27182338,8.381627e+00,NaN,"['ENSG00000111790', 'ENSG00000152944', 'ENSG00...",[],27181838,27182838,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...,0
7788,chr12,VEZT,95696030,8.337806e+00,NaN,NaN,[],95695530,95696530,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...,0


In [87]:
disorders = pd.read_csv("mendelian_disorders.csv", delimiter=';', header=None)
disorders = disorders.applymap(lambda s: s.lower().replace('-', ' ').replace('_', ' ').strip())
disorders

,0,1
0,age at first birth,age of menarche
1,age at first birth,intellectual disability
2,age at first birth,fertility
3,age at first birth,hypogonadism
4,age at menarche,precocious puberty
...,...,...
601,ulcerative colitis,autoantibody
602,ulcerative colitis,recurrent infection
603,ulcerative colitis,intestinal ulceration
604,ulcerative colitis,ulcerative colitis


In [88]:
from collections import defaultdict

disorders_map = defaultdict(list)
for _, row in disorders.iterrows():
    trait = row[0]
    mendelian_disorder = row[1]
    disorders_map[trait].append(mendelian_disorder)
disorders_map

defaultdict(list,
            {'age at first birth': ['age of menarche',
              'intellectual disability',
              'fertility',
              'hypogonadism'],
             'age at menarche': ['precocious puberty',
              'amenorrhea',
              'polycystic ovarian syndrome',
              'pcos',
              'oligomenorrhea',
              'low estradiol',
              'endocrinological dysfunction',
              'hypothalamic dysfunction',
              'hypothalamic dysregulation',
              'hypogonadism',
              'premature ovarian failure',
              'premature menarche',
              'pituitary dysfunction',
              'pituitary dysregulation',
              'sexual precocity',
              'rapid virilization',
              'decreased fertility',
              'increased fertility'],
             'age at menopause': ['amenorrhea',
              'polycystic ovarian syndrome',
              'pcos',
              'oligomenorrhea',
  

In [89]:
disorders_map.keys()

dict_keys(['age at first birth', 'age at menarche', 'age at menopause', 'age related macular degeneration', 'alzheimers disease', 'anxiety', 'autism', 'bipolar disorder', 'birthweight', 'body fat percentage', 'body mass index', 'bone mineral density', 'celiac disease', 'childhood body mass index', 'coronary artery disease', 'crohns disease', 'depression', 'education', 'extra glomerular filtration rate (creatinine)', 'fasting insulin', 'height', 'hemoglobin', 'hemoglobin a1c', 'homa b', 'inflammatory bowel disease', 'insulin resistance & fasting glucose', 'kidney disease', 'leptin', 'lupus', 'major depression', 'microalbuminuria', 'multiple sclerosis', 'neuroticism', 'packed cell volume', 'platelet count', 'primary biliary cirrhosis', 'red blood cell count', 'resting heart rate', 'rheumatoid arthritis', 'schizophrenia', 'serum urate concentration', 'subjective sense of well being', 'two hour fasting glucose', 'type 1 diabetes', 'type 2 diabetes', 'ulcerative colitis'])

In [108]:
def check_mendelian(lst):
    for el in lst:
        for trait in disorders_map.keys():
                if el in trait or trait in el:
                    print(el, "|", trait)
                    return disorders_map[trait]
    return None
            

In [109]:
df['mendelian_disorders'] = df['DISEASE/TRAIT'].apply(check_mendelian)

hemoglobin concentration | hemoglobin
body mass index | body mass index
chronic obstructive pulmonary disease or resting heart rate (pleiotropy) | resting heart rate
systemic lupus erythematosus | lupus
platelet count | platelet count
schizophrenia | schizophrenia
height | height
systemic lupus erythematosus | lupus
platelet count | platelet count
type 2 diabetes (adjusted for bmi) | type 2 diabetes
height | height
body mass index | body mass index
macrovascular complications in type 2 diabetes | type 2 diabetes
height | height
waist circumference adjusted for body mass index | body mass index
coronary artery disease | coronary artery disease
mean corpuscular hemoglobin | hemoglobin
red blood cell count | red blood cell count
neutrophil level response to clozapine in treatment resistant schizophrenia | schizophrenia
type 2 diabetes | type 2 diabetes
coronary artery disease | coronary artery disease
schizophrenia | schizophrenia
mean corpuscular hemoglobin | hemoglobin
schizophrenia | s

In [110]:
df[~df['mendelian_disorders'].isna()]

,Chr_x,Gene symbol,loc,z-score,int_size,eqtl_genes,DISEASE/TRAIT,l_break,r_break,seq,target,mendelian_disorders
91,chr20,-,39797465,24077.961325,NaN,"['ENSG00000132793', 'ENSG00000124177', 'ENSG00...","[hemoglobin concentration, birth weight]",39796965,39797965,CCACAGCCAGGGGACCTGGAAGGAAGTTTTTCTTTCCACTTCACAG...,1,"[small red blood cells, microcytic anemia, mac..."
310,chr5,CTC-467M3.1,88016008,3788.411519,NaN,"['ENSG00000271904', 'ENSG00000248309']",[body mass index],88015508,88016508,AGATAGACCAAGATTGTTTACAGAAAGTGACTCATCAAAATCAAAA...,1,"[overweight, central obesity, visceral fat, le..."
727,chr2,TNS1,218667372,1743.106556,NaN,"['ENSG00000231672', 'ENSG00000079308', 'ENSG00...","[intraocular pressure, chronic obstructive pul...",218666872,218667872,AAATATCTCCTATCTTGGAAGATAATTGGGGTTTGCTAGGGAATGA...,1,"[tachycardia, bradycardia, cardiac arrhythmia,..."
762,chr11,IRF7,614318,1634.105539,NaN,"['ENSG00000174775', 'ENSG00000069696', 'ENSG00...",[systemic lupus erythematosus],613818,614818,ACAGGTACGGCTCTGCCTGGTGCGGGGACTCTGGGGCCGCGGCCTC...,1,"[joint pain, joint damage, degenerative joint ..."
838,chr15,RP11-244F12.2,63363490,1416.416032,NaN,"['ENSG00000138613', 'ENSG00000103642']",[platelet count],63362990,63363990,CCACGGCACACCGTCAGTGAACCTTCACCAAACCCCACGTGCATTT...,1,"[disseminated intravascular coagulation, dic, ..."
898,chr7,TTYH3,2702816,1278.881607,116.0,['ENSG00000136295'],[schizophrenia],2702316,2703316,CGGCTGTGAACATCCCTCAGCCCCTGCTGTCCCCCCTTGGGACTAA...,1,"[paranoia, paranoid delusions, psychosis, hall..."
1007,chr19,CTC-435M10.3,41930396,961.053210,NaN,"['ENSG00000142046', 'ENSG00000177191', 'ENSG00...",[height],41929896,41930896,TTAAAAAAAAAAAAAAGCTGGTTTGAGGATTAGGTGGCAAACCGTG...,1,"[undergrowth, short stature, small for gestati..."
1285,chr12,-,129278864,640.578487,NaN,"['ENSG00000139370', 'ENSG00000151948']","[systemic lupus erythematosus, systemic lupus ...",129278364,129279364,GCAGGTGATACTATTTGTTTAAGAAACTGGGATGCCAACTAACACG...,1,"[joint pain, joint damage, degenerative joint ..."
1441,chr11,ETS1,128330520,607.525245,NaN,['ENSG00000134954'],"[platelet count, plateletcrit, platelet count]",128330020,128331020,ACACATGCACACATTCACACACACACCTTTTGCCAGTGCCCAACAA...,1,"[disseminated intravascular coagulation, dic, ..."
1536,chr17,ZZEF1,3947644,583.428453,NaN,"['ENSG00000167740', 'ENSG00000185722', 'ENSG00...","[type 2 diabetes (adjusted for bmi), type 2 di...",3947144,3948144,CTTACACCTCATTCCGCCTCAAATTACAGTTACCTGCACCTGTGTC...,1,"[noninsulin dependent diabetes, pancreatic abn..."


In [111]:
df.to_excel("output.xlsx", sheet_name='results')  